# Gaussian OpenPulse waveform generator

This notebook shows how to use `qbraid_algorithms.openpulse` to generate an **OpenQASM 3 + OpenPulse** program that defines a **Gaussian** waveform and a `defcal` that plays it on a target qubit.

In [2]:
import pyqasm

from qbraid_algorithms.openpulse import GaussianPulse, PulseParams, generate_program


## Generate a standalone Gaussian pulse QASM program

First we define the parameters of the pulse we want to play on a target qubit.

In [12]:
pulse = GaussianPulse(amplitude=1.0 + 2.0j, duration="16ns", sigma="4ns")
params = PulseParams(
    frame_frequency=5.0e9,
    frame_phase=0.0,
    port_name="d0",
    frame_name="driveframe",
    waveform_name="wf",
    defcal_name="play_gaussian",
    qubit=0,
)

Then we pass the parameters to a `generate_program()` call to load a PyQASM module that defines and plays the pulse.

In [13]:
module = generate_program(pulse=pulse, params=params)

After loading the program as a PyQASM module, we can run all the standard PyQASM operations on it, such as displaying it:

In [14]:
print(pyqasm.dumps(module))

OPENQASM 3.0;
defcalgrammar "openpulse";
cal {
    port d0;
    frame driveframe = newframe(d0, 5000000000.0, 0.0);
    waveform wf = gaussian(1.0 + 2.0im, 16ns, 4ns);
}
defcal play_gaussian() $0 {
    play(driveframe, wf);
}



or unrolling it:

In [15]:
unrolled = module.copy()
unrolled.unroll()

print(pyqasm.dumps(unrolled))

OPENQASM 3.0;
qubit[1] __PYQASM_QUBITS__;
defcalgrammar "openpulse";
cal {
 port d0;
 frame driveframe = newframe(d0, 5000000000.0, 0.0, 0ns);
 waveform wf = gaussian(1.0 + 2.0im, 16.0ns, 4.0ns);
}
defcal play_gaussian() $0 {
 play(driveframe, wf);
}



## Example: Qubit Spectroscopy

Qubit spectroscopy is a common pulse-level experiment where the qubit is driven with the same pulse
while the drive frequency is swept. By observing the qubit’s response as a function of frequency,
its transition frequency can be identified.

In this example, we use the Gaussian OpenPulse generator to define a reusable pulse waveform and a
`defcal` routine that plays it on a drive frame. This calibration is then called inside a frequency
sweep loop, showing how the generator can be used as a building block for larger pulse-level
experiments.

In [18]:
import pyqasm

base_module = generate_program(pulse=pulse, params=params)
base_qasm = pyqasm.dumps(base_module).rstrip()

qubit_spectroscopy_qasm_code = base_qasm + f"""

const float frequency_start = 4.5e9;
const float frequency_step = 1e6;
const int frequency_num_steps = 3;

cal {{
   set_frequency({params.frame_name}, frequency_start);
}}

for int i in [1:frequency_num_steps] {{
   cal {{
       shift_frequency({params.frame_name}, frequency_step);
   }}
   {params.defcal_name} ${params.qubit};
   measure ${params.qubit};
}}
"""

print(qubit_spectroscopy_qasm_code)

OPENQASM 3.0;
defcalgrammar "openpulse";
cal {
    port d0;
    frame driveframe = newframe(d0, 5000000000.0, 0.0);
    waveform wf = gaussian(1.0 + 2.0im, 16ns, 4ns);
}
defcal play_gaussian() $0 {
    play(driveframe, wf);
}

const float frequency_start = 4.5e9;
const float frequency_step = 1e6;
const int frequency_num_steps = 3;

cal {
   set_frequency(driveframe, frequency_start);
}

for int i in [1:frequency_num_steps] {
   cal {
       shift_frequency(driveframe, frequency_step);
   }
   play_gaussian $0;
   measure $0;
}



In [19]:
qubit_spectroscopy_module = pyqasm.loads(qubit_spectroscopy_qasm_code)
qubit_spectroscopy_module.unroll()
print(pyqasm.dumps(qubit_spectroscopy_module))

OPENQASM 3.0;
qubit[1] __PYQASM_QUBITS__;
defcalgrammar "openpulse";
cal {
 port d0;
 frame driveframe = newframe(d0, 5000000000.0, 0.0, 0ns);
 waveform wf = gaussian(1.0 + 2.0im, 16.0ns, 4.0ns);
}
defcal play_gaussian() $0 {
 play(driveframe, wf);
}
cal {
 set_frequency(driveframe, 4500000000.0);
}
cal {
 shift_frequency(driveframe, 4501000000.0);
}
play_gaussian __PYQASM_QUBITS__[0];
measure __PYQASM_QUBITS__[0];
cal {
 shift_frequency(driveframe, 4502000000.0);
}
play_gaussian __PYQASM_QUBITS__[0];
measure __PYQASM_QUBITS__[0];
cal {
 shift_frequency(driveframe, 4503000000.0);
}
play_gaussian __PYQASM_QUBITS__[0];
measure __PYQASM_QUBITS__[0];

